<a href="https://colab.research.google.com/github/luislizcano/seagrass-mapping-tb/blob/main/Preprocessing/NDTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtain NDTI of specific areas of interest to help the imagery preselection for seagrass mapping

This script classify seagrass beds in selected BOA images using ground-data to train the Support Vector Machine classifier. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>


Script by: Luis Lizcano-Sandoval (2021)<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu
Updated: 08/05/2022

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds and land
3. Apply Depth-Invariant Index (generates band-ratios B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (SVM)
6. Get confusion matrices and accuracies
7. Export classification images to EE Assets (.tiff)
8. Save matrices in GDrive (.xlxs)

In [1]:
## Authenticate your EE account
import ee
ee.Authenticate()

ee.Initialize(project='earth-engine-252816')

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass-mapping-tb.git

In [ ]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,'/content/seagrass-mapping-tb')
sys.path.append('/content/seagrass-mapping-tb/Preprocessing/')

## Verify you loaded the EE module correctly:
from process import start_processing
print('EE version: ',ee.__version__)

## Import collections and data

In [ ]:
## Load collection (use level-2; but B10 wont be available)
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
## FAO GAUL: Global Administrative Unit Layers:
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
## Tampa Bay segments
tb_segments = ee.FeatureCollection("users/lizcanosandoval/Seagrass/TBEP_Segments_Adapted")

## Settings

In [ ]:
## Check and set the following parameters:
year = '2023'
regionCode = 'MRTC'
tileName = ['17RLL']
countryName = 'United States of America'

## Some other settings:
fileName = 'Sentinel-2_Means_'+regionCode+'_'+year
## Imagery parameters:
start_date = year+'-01-01'
end_date = year+'-12-31'
cloud_percent = 40


In [ ]:
## Select country
## To select country by GAUL see: http://www.fao.org/countryprofiles/iso3list/en/
land = countries.filter(ee.Filter.eq('ADM0_NAME',countryName))

## Select segment
segment = tb_segments.filter(ee.Filter.stringContains('name',regionCode))
segmentName = segment.aggregate_array('name')
print(segmentName.get(0).getInfo()+' - '+countryName+' ('+year+')')

## RESTART RUNTIME

In [ ]:
os.kill(os.getpid(), 9)

**Load required libraries:**

In [ ]:
import os, sys #Import os, sys again if runtime was restarted.
sys.path.insert(0,'/content/seagrass_scripts')
sys.path.append('/content/seagrass_scripts/bin/')

from google.colab import auth
auth.authenticate_user()

import google
SCOPES = ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/earthengine']
CREDENTIALS, project_id = google.auth.default(default_scopes=SCOPES)

import ee
ee.Initialize(CREDENTIALS, project='earth-engine-252816')

In [ ]:
## Verify you loaded the EE module correctly:
import functions
print('EE version: ',ee.__version__)

In [ ]:
## Load collection (use level-2; but B10 wont be available)
collection = sentinel2.filterDate(start_date, end_date)\
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percent))\
              .filterBounds(region)\
              .filter(ee.Filter.inList('MGRS_TILE',tile_list)).sort('MGRS_TILE')

tile_list = ee.List(collection.aggregate_array('MGRS_TILE')).distinct().sort()
print('Images Found:',collection)
print('Tiles Found:',tile_list)

**Run functions:**

In [ ]:
## Rescale collection
scaleColl = collection.map(functions.rescale())

bands = ['B5','NDTI']
newName = ['704','NDTI']

## Apply cloud and land mask.
maskedColl = scaleColl.map(maskLand(land)) # Mask Land
maskedColl = maskedColl.map(CloudMask) # Mask Clouds
maskedColl = maskedColl.map(clipRegion(segment)) # Mask by Regions
maskedColl = maskedColl.map(NDTI) # Get NDTI
maskedColl = maskedColl.select(bands, newName) # Select bands and rename

## Get the mean and std values from bands of interest.
statsColl = maskedColl.map(getStats)

## Count the number of valid pixels from bands of interest (only one band is ok).
countColl = maskedColl.map(countPixels)

## Use an equals filter to specify how the collections match.
var toyFilter = ee.Filter.equals(**{
  'leftField': 'system:index',
  'rightField': 'system:index'})
## Define the join.
innerJoin = ee.Join.inner('stats','counts')
## Apply the join.
toyJoin = innerJoin.apply(statsColl, countColl, toyFilter)

## Final collection with all the properties together by feature
endColl = toyJoin.map(functions.cleanJoin())


**Extract properties:**

In [ ]:
## Extract a list of property values from the 'statsCollfeat' collection, individually.
system_id = ee.List(endColl.aggregate_array('system:index'))\
                .map(lambda x: ee.String(x).slice(0,38))
tile_id = ee.List(endColl.aggregate_array('MGRS_TILE'))
time_id = ee.List(endColl.aggregate_array('GENERATION_TIME')\
              .map(lambda x: ee.Date(ee.Number(x)).format('yyyy-MM-dd')))
zenith = ee.List(endColl.aggregate_array('MEAN_SOLAR_ZENITH_ANGLE'))
azimuth = ee.List(endColl.aggregate_array('MEAN_SOLAR_AZIMUTH_ANGLE'))
spacecraft = ee.List(endColl.aggregate_array('SPACECRAFT_NAME'))
valid_pixels = ee.List(endColl.aggregate_array('Valid_Pixels'))
cloud_cover = ee.List(endColl.aggregate_array('CLOUDY_PIXEL_PERCENTAGE'))
get704_mean = ee.List(endColl.aggregate_array('704_mean'))
get704_stdv = ee.List(endColl.aggregate_array('704_stdDev'))
getNdti_mean = ee.List(endColl.aggregate_array('NDTI_mean'))
getNdti_stdv = ee.List(endColl.aggregate_array('NDTI_stdDev'))

Hereinafter, the computation becomes more complex and the outputs do not print in the console (ERROR: USER MEMORY LIMIT EXCEEDED), but the process will continue to export tables.

In [ ]:
## Pair lists
pairedLists = ee.List.sequence(0,get704_mean.length().subtract(1),1).map(lambda i:\
    [system_id.get(i),tile_id.get(i),time_id.get(i),zenith.get(i),azimuth.get(i),\
     spacecraft.get(i),valid_pixels.get(i),cloud_cover.get(i),get704_mean.get(i),\
     get704_stdv.get(i),getNdti_mean.get(i),getNdti_stdv.get(i)])

## Convert the list of properties to a feature
## that will be set as a new column.
def tupleToFeature(feat):
  tuple = ee.List(feat)

  return ee.Feature(None, {\
    image_id: tuple.get(0),\
    tile_id: tuple.get(1),\
    time_id: tuple.get(2),\
    zenith: tuple.get(3),\
    azimuth: tuple.get(4),\
    spacecraft: tuple.get(5),\
    valid_pixels: tuple.get(6),\
    cloud_cover: tuple.get(7),\
    mean_704: tuple.get(8),\
    stdv_704: tuple.get(9),\
    mean_ndti: tuple.get(10),\
    stdv_ndti: tuple.get(11)})

## Run the function to pair lists
endList = pairedLists.map(tupleToFeature)
#print('endList',endList);

## Convert list of features to a feature collection. Required to export it as .csv
endTable = ee.FeatureCollection(endList)

**Start classification loop:**

In [ ]:
%%time
start_processing(imageSource,satellite,regionName,boaFolder,exportFolder,dataFolder,smoothStr,nameCode,
                 regionCountry,state,imageList,sand_areas,groundPoints,land,regions,1,1,0,0)